<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://gitlab.com/AkashNarayanan" target="_blank"> Akash Narayanan B</a></td>    
</table>
<hr>

So far, we have learnt about formulating combinatorial optimization problems as QUBO or Ising Model problems. We have also learnt how to convert between these two formulations. Our aim is to take advantage of Quantum Computers to solve these problems. To do so, we have to formulate our QUBO or Ising Model problems in a way that they can be run on a Quantum Computer.

The [Ocean SDK](https://github.com/dwavesystems/dwave-ocean-sdk) provides us many open-source tools to aid us in the problem solving process. Now let's take a look at the `BinaryQuadraticModel` class available in the `dimod` package of the Ocean SDK.

# Binary Quadratic Model (BQM)

`BinaryQuadraticModel` class helps us to formulate our QUBO or Ising Model problems into a form suitable to be run on a Quantum Computer. Let us quickly recall the objective functions of QUBO and Ising Model.

The objective function of a QUBO is,

$$f(x) = \sum\limits_{i}^{N} {Q_{i, i} x_i} + \sum\limits_{i < j}^{N} {Q_{i, j} x_i x_j} \qquad\qquad x_i\in \{0,1\}$$

where the variables can take the values $0$ and $1$.

The objective function of an Ising Model is,

$$E(s) = \sum\limits_{i=1}^N h_i s_i + \sum\limits_{i<j}^N J_{i,j} s_i s_j   \qquad\qquad s_i \in\{-1,+1\}$$

where the variables can take the values $-1$ and $+1$ corresponding to the physical Ising spins.

The objective function of a Binary Quadratic Model is,

$$E(v) = \sum\limits_{i=1} a_i v_i + \sum\limits_{i<j} b_{i,j} v_i v_j + c \qquad\qquad v_i \in \{0,1\} \text{  or } \{-1,+1\}$$

We can notice that the variable $v_i$ can correspond either to $\{0, 1\}$ or to the physical Ising spins $\{-1, +1\}$. This way a BQM can conveniently represent both a QUBO and an Ising Model.

# Creating an Instance of BQM

Let us first take a look at some of the essential parameters required to create an instance of the `BinaryQuadraticModel` class.

## Parameters

- `linear` 
   - The linear terms of the objective function should be defined as a dictionary.
   - The keys of the dictionary should be the variables and their respective values should be the coefficients associated with these variables. For example,

    ```python
    {'x1': 3, 'x2': 5, 'x3': 4, 'x4': 7}
    ```
    
- `quadratic`
   - The quadratic terms of the objective function should be defined as a dictionary.
   - The keys of the dictionary should be the pairs of variables defined as tuples and their respective values should be the coefficients associated with these pairs of variables. For example,

    ```python
    {('x1', 'x2'): 2, ('x2', 'x3'): 5}
    ```
                   
- `offset`
    - Constant energy offset value associated with the BQM can be set using this parameter. 
    - If there is no offset, it can be just set to `0`.
    
- `Vartype`
    - This parameter sets the variable type of the BQM. To create a QUBO instance, set this parameter to `'BINARY'`.
    - To create an Ising Model instance, set this parameter to `'SPIN'`.
    
## Example

Let us now try to create a BQM instance for the following objective function,

$$f(x_1, x_2, x_3, x_4) = - 5x_1 - 3x_2 - 8x_3 - 6x_4 + 4x_1 x_2 + 8x_1 x_3 + 2x_2 x_3 + 10x_3 x_4$$

We should define the linear and quadratic parts of the objective function as dictionaries and pass it as `linear` and `quadratic` arguments. In the objective function,

- The linear part is $- 5x_1 - 3x_2 - 8x_3 - 6x_4$. The corresponding dictionary can be defined as 

```python
{'x1': -5, 'x2': -3, 'x3': -8, 'x4': -6}
```
                       
- The quadratic part is $4x_1 x_2 + 8x_1 x_3 + 2x_2 x_3 + 10x_3 x_4$. The corresponding dictionary can be defined as

```python
{('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}
```  

- There is no offset, so we can set the `offset` parameter to `0`.

We can create a QUBO instance of BQM by setting the `Vartype` parameter to `'BINARY'`.

In [1]:
from dimod import BinaryQuadraticModel

linear = {'x1': -5, 'x2': -3, 'x3': -8, 'x4': -6}
quadratic = {('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}
offset = 0
vartype = 'BINARY'

bqm_qubo = BinaryQuadraticModel(linear, quadratic, offset, vartype)

We can create an Ising instance of BQM for the same objective function by setting the `Vartype` parameter to `'SPIN'`.

In [2]:
vartype = 'SPIN'

bqm_ising = BinaryQuadraticModel(linear, quadratic, offset, vartype)

<div class="alert alert-block alert-info">
Note that the <code>Vartype</code> parameter just sets the variable type for a BQM and doesn't automatically convert between a QUBO and an Ising Model. In the above example, <code>bqm_qubo</code> and <code>bqm_ising</code> are two instances of BQM created from the same objective function. However <code>bqm_qubo</code> and <code>bqm_ising</code> are not equal. Recall that in order to convert a QUBO to an Ising Model, the following transformation should be used.

$$ x_j = \frac{s_j + 1}{2} $$
    
There are methods in the Ocean SDK for converting between the formulations. We will learn about them later on.
</div>

## Attributes

We can check the values assigned to the parameters using the attributes of the BQM class.

```python
>>> bqm_qubo.linear
{'x1': -5.0, 'x2': -3.0, 'x3': -8.0, 'x4': -6.0}

>>> bqm_qubo.quadratic
{('x1', 'x2'): 4, ('x1', 'x3'): 8, ('x2', 'x3'): 2, ('x3', 'x4'): 10}

>>> bqm_qubo.offset
0

>>> bqm_qubo.vartype
<Vartype.BINARY: frozenset({0, 1})>

>>> bqm_ising.vartype
<Vartype.SPIN: frozenset({1, -1})>
```

These attributes are helpful to probe the details of an instance of BQM. The above two instances `bqm_qubo` and `bqm_ising` share the same attribute values except for the `vartype` attribute.

### Task 1

Create a QUBO instance of BQM for the following objective function

$$f(x_1, x_2) = 5x_1 + 7x_1 x_2 - 3x_2$$

[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-1)

### Task 2

Create an Ising instance of BQM for the following objective function

$$f(s_1, s_2, s_3, s_4) = s_1 + s_2 + s_3 + s_4 - 6s_1 s_3 - 6s_1 s_4 - 6s_3 s_4 - 6s_1 s_2$$

[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-2)

### Task 3

Find the `linear`, `quadratic`, `offset` and `vartype` values of the following instance of BQM.

In [3]:
%run bqm_functions.py

bqm_mystery = BinaryQuadraticModel(linear, quadratic, offset, vartype)

# Enter your code here





[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-3)

# Finding the Lowest Energy Using a Classical Sampler

Ocean SDK provides classical, quantum and hybrid samplers to help us find optimal solutions to our problems. A sampler tries to sample low energy states for a given BQM and returns an iterable of samples in the ascending order of the energy values.

We are going to use `ExactSolver()` to classically sample our problems. It works by finding the energy values of all the possible samples for a given BQM. As you can guess, this is not an efficient process but it is good enough for small problems. The general limit is 18 variables beyond which the process becomes very slow. `ExactSolver()` can be helpful to test our code during development.

## Example: QUBO Instance

Let us try to create a QUBO instance of BQM and find the energy values for the following objective function

$$f(x_1, x_2, x_3, x_4) = 3x_1 - 7x_2 + 11x_3 - x_4 + 9x_1 x_2 + x_1 x_3 + 2x_2 x_3 + 8x_3 x_4$$

In the objective function,

- The linear part is $3x_1 - 7x_2 + 11x_3 - x_4$
- The quadratic part is $9x_1 x_2 + x_1 x_3 + 2x_2 x_3 + 8x_3 x_4$

In [4]:
from dimod import BinaryQuadraticModel
from dimod.reference.samplers import ExactSolver

linear = {'x1': 3, 'x2': -7, 'x3': 11, 'x4': -1}
quadratic = {('x1', 'x2'): 9, ('x1', 'x3'): 1, ('x2', 'x3'): 2, ('x3', 'x4'): 8}
offset = 0
vartype = 'BINARY'

bqm_qubo = BinaryQuadraticModel(linear, quadratic, offset, vartype)

Now that we have created a QUBO instance, we can then assign `ExactSolver()` to a variable for convenience. Then we should pass the instance `bqm_qubo` as an argument to the `sample()` method of `ExactSolver()` and assign it to another variable. This variable would then contain all the possible samples in the ascending order of their energy values.

In [5]:
sampler = ExactSolver()
sampleset = sampler.sample(bqm_qubo)

print(sampleset)

   x1 x2 x3 x4 energy num_oc.
12  0  1  0  1   -8.0       1
3   0  1  0  0   -7.0       1
15  0  0  0  1   -1.0       1
0   0  0  0  0    0.0       1
14  1  0  0  1    2.0       1
1   1  0  0  0    3.0       1
13  1  1  0  1    4.0       1
2   1  1  0  0    5.0       1
4   0  1  1  0    6.0       1
7   0  0  1  0   11.0       1
11  0  1  1  1   13.0       1
6   1  0  1  0   15.0       1
8   0  0  1  1   18.0       1
5   1  1  1  0   19.0       1
9   1  0  1  1   22.0       1
10  1  1  1  1   26.0       1
['BINARY', 16 rows, 16 samples, 4 variables]


In the above output,

- First column represents the serial number
- The next four columns represent the different values for the four variables present in the objective function
- `energy` column refers to the value of the objective function for each sample
- `num_oc .` refers to the number of occurences for each sample. Since the classical sampler exactly determines the energy value for each and every sample, number of occurence for each sample is just 1.

We can observe from the output that the first sample `(0, 1, 0, 1)` minimizes the objective function to a value of `-8.0`. That is the optimal solution we are looking for! The energy values of the subsequent samples are `-7.0`, `-1.0` and so on and so forth.

### Accessing a single optimal solution

The optimal solution that produces the lowest energy value can be accessed using the `first` attribute.

```python
>>> print(sampleset.first)
Sample(sample={'x1': 0, 'x2': 1, 'x3': 0, 'x4': 1}, energy=-8.0, num_occurrences=1)
```

If we want just the sample that produces the lowest energy, we can access it using the `sample` attribute.

```python
>>> print(sampleset.first.sample)
{'x1': 0, 'x2': 1, 'x3': 0, 'x4': 1}
```

If we want just the energy value of the optimal solution, we can access it using the `energy` attribute.

```python
>>> print(sampleset.first.energy)
-8.0
```

### Accessing multiple optimal solutions

Incase our problem has multiple samples that produce the lowest energy value, we can access all of those optimal solutions at once using the `lowest()` method. For example, let's consider the sample set of a random Ising instance that has multiple optimal solutions.  

```python
>>> sampleset_random = sampler.sample(bqm_random)
>>> print(sampleset_random)
```
```
   0  1  2 energy num_oc.
0 -1 -1 -1   -3.0       1
5 +1 +1 +1   -3.0       1
1 +1 -1 -1    1.0       1
2 +1 +1 -1    1.0       1
3 -1 +1 -1    1.0       1
4 -1 +1 +1    1.0       1
6 +1 -1 +1    1.0       1
7 -1 -1 +1    1.0       1
['SPIN', 8 rows, 8 samples, 3 variables]
```

In the above sample set we can observe that there are two optimal solutions that produce the lowest energy of `-3.0`. Using the `first` attribute here would display only one of those two optimal solutions.

```python
>>> print(sampleset_random.first)
Sample(sample={0: -1, 1: -1, 2: -1}, energy=-3.0, num_occurrences=1)
```

Whereas the `lowest()` method would display both the optimal solutions.

```python
>>> print(sampleset_random.lowest())
```
```
   0  1  2 energy num_oc.
0 -1 -1 -1   -3.0       1
1 +1 +1 +1   -3.0       1
['SPIN', 2 rows, 2 samples, 3 variables]
```


### Task 4

Find the optimal sample of the QUBO instance that produces the lowest energy value for the objective function used in Task 1.

$$f(x_1, x_2) = 5x_1 + 7x_1 x_2 - 3x_2$$

[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-4)

## Example: Ising Instance

Let us now try to create an Ising instance and find the optimal solution for the following objective function.

$$f(s_1, s_2, s_3, s_4) = 3s_1 - 7s_2 + 11s_3 - s_4 + 9s_1 s_2 + s_1 s_3 + 2s_2 s_3 + 8s_3 s_4$$


In [6]:
linear = {'s1': 3, 's2': -7, 's3': 11, 's4': -1}
quadratic = {('s1', 's2'): 9, ('s1', 's3'): 1, ('s2', 's3'): 2, ('s3', 's4'): 8}
offset = 0
vartype = 'SPIN'

bqm_ising = BinaryQuadraticModel(linear, quadratic, offset, vartype)

sampler = ExactSolver()
sampleset = sampler.sample(bqm_ising)

print(sampleset)

   s1 s2 s3 s4 energy num_oc.
12 -1 +1 -1 +1  -40.0       1
3  -1 +1 -1 -1  -22.0       1
13 +1 +1 -1 +1  -18.0       1
14 +1 -1 -1 +1  -18.0       1
4  -1 +1 +1 -1  -14.0       1
15 -1 -1 -1 +1   -4.0       1
1  +1 -1 -1 -1    0.0       1
2  +1 +1 -1 -1    0.0       1
11 -1 +1 +1 +1    0.0       1
6  +1 -1 +1 -1    4.0       1
5  +1 +1 +1 -1   12.0       1
0  -1 -1 -1 -1   14.0       1
7  -1 -1 +1 -1   14.0       1
9  +1 -1 +1 +1   18.0       1
10 +1 +1 +1 +1   26.0       1
8  -1 -1 +1 +1   28.0       1
['SPIN', 16 rows, 16 samples, 4 variables]


In [7]:
print(sampleset.first)

Sample(sample={'s1': -1, 's2': 1, 's3': -1, 's4': 1}, energy=-40.0, num_occurrences=1)


Therefore the sample `{'s1': -1, 's2': 1, 's3': -1, 's4': 1}` minimizes the objective function to an energy value of `-40`.

### Task 5

Find the optimal sample of the Ising instance that produces the lowest energy value for the objective function used in Task 2.

$$f(s_1, s_2, s_3, s_4) = s_1 + s_2 + s_3 + s_4 - 6s_1 s_3 - 6s_1 s_4 - 6s_3 s_4 - 6s_1 s_2$$

[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-5)

### Task 6

Create a QUBO instance of BQM for the given objective function and find the optimal solution.

$$f(x_1, x_2, x_3, x_4) = 3x_1 - x_2 + 10x_3 + 7x_4 + 2x_1 x_2 - 5x_1 x_3 + 3x_2 x_3 + 11x_3 x_4$$

[Click Here for Solution](BQM_Formulation_Solution.ipynb#Task-6)

# References

1. ["Binary Quadratic Models"](https://test-projecttemplate-dimod.readthedocs.io/en/latest/reference/bqm/index.html), `dimod` Documentation, accessed August, 2021.
2. ["Classical Solvers"](https://docs.ocean.dwavesys.com/en/stable/overview/cpu.html#), D-Wave Ocean Software Documentation, accessed August 2021.
3. ["Exact Solver"](https://docs.ocean.dwavesys.com/en/stable/docs_dimod/reference/sampler_composites/samplers.html#exact-solver), D-Wave Ocean Software Documentation, accessed August 2021.